# Document Processing Tool

## Naive Approach

In [1]:
class Paragraph:
    def __init__(self, text):
        self.text = text

    def render_html(self):
        return f"<p>{self.text}</p>"

    def count_words(self):
        return len(self.text.split())

    def validate(self):
        return len(self.text.strip()) > 0

    def apply_theme(self, theme):
        print(f"Applying '{theme}' theme to paragraph.")

class Image:
    def __init__(self, filename):
        self.filename = filename

    def render_html(self):
        return f"<img src='{self.filename}' />"

    def count_words(self):
        return 0  # Images don't have words

    def validate(self):
        return self.filename.endswith(('.png', '.jpg'))

    def apply_theme(self, theme):
        print(f"Applying '{theme}' theme to image.")

class Table:
    def __init__(self, rows):
        self.rows = rows

    def render_html(self):
        html = "<table>"
        for row in self.rows:
            html += "<tr>" + "".join(f"<td>{cell}</td>" for cell in row) + "</tr>"
        html += "</table>"
        return html

    def count_words(self):
        return sum(len(cell.split()) for row in self.rows for cell in row)

    def validate(self):
        return all(isinstance(row, list) for row in self.rows)

    def apply_theme(self, theme):
        print(f"Applying '{theme}' theme to table.")

if __name__ == "__main__":
    doc = [
        Paragraph("This is a paragraph."),
        Image("photo.jpg"),
        Table([["Name", "Age"], ["Alice", "30"]])
    ]

    for element in doc:
        print(element.render_html())
        element.apply_theme("Dark")
        print("Word count:", element.count_words())
        print("Valid?", element.validate())
        print("-" * 40)

<p>This is a paragraph.</p>
Applying 'Dark' theme to paragraph.
Word count: 4
Valid? True
----------------------------------------
<img src='photo.jpg' />
Applying 'Dark' theme to image.
Word count: 0
Valid? True
----------------------------------------
<table><tr><td>Name</td><td>Age</td></tr><tr><td>Alice</td><td>30</td></tr></table>
Applying 'Dark' theme to table.
Word count: 4
Valid? True
----------------------------------------


## Visitor Pattern

In [2]:
# Define Visitor Interface

from abc import ABC, abstractmethod

class Visitor(ABC):

    @abstractmethod
    def visit_paragraph(self, element):
        pass

    @abstractmethod
    def visit_image(self, element):
        pass

    @abstractmethod
    def visit_table(self, element):
        pass

In [3]:
#Define the Element Interface

class DocumentElement(ABC):
    @abstractmethod
    def accept(self, visitor: Visitor):
        pass

In [4]:
#Implement Concrete Elements

class Paragraph(DocumentElement):
    def __init__(self, text):
        self.text = text

    def accept(self, visitor: Visitor):
        visitor.visit_paragraph(self)

class Image(DocumentElement):

    def __init__(self, filename):
        self.filename = filename

    def accept(self, visitor: Visitor):
        visitor.visit_image(self)

class Table(DocumentElement):

    def __init__(self, rows):
        self.rows = rows

    def accept(self, visitor: Visitor):
        visitor.visit_table(self)

In [5]:
# Implement Comcrete Visitors

class HTMLRenderer(Visitor):
    def visit_paragraph(self, element):
        print(f"<p>{element.text}</p>")

    def visit_image(self, element):
        print(f"<img src='{element.filename}' />")

    def visit_table(self, element):
        print("<table>")
        for row in element.rows:
            print("  <tr>" + "".join(f"<td>{cell}</td>" for cell in row) + "</tr>")
        print("</table>")

In [6]:
# Client Code

if __name__ == "__main__":
    doc = [
        Paragraph("Hello world."),
        Image("banner.png"),
        Table([["Name", "Age"], ["Alice", "30"], ["Bob", "25"]])
    ]

    renderer = HTMLRenderer()

    for element in doc:
        element.accept(renderer)

<p>Hello world.</p>
<img src='banner.png' />
<table>
  <tr><td>Name</td><td>Age</td></tr>
  <tr><td>Alice</td><td>30</td></tr>
  <tr><td>Bob</td><td>25</td></tr>
</table>


### Scaling up: Add a word counter visitor

In [10]:
class WordCounter(Visitor):

    def __init__(self):
        self.word_count = 0

    def visit_paragraph(self, element):
        self.word_count += len(element.text.split())

    def visit_image(self, element):
        pass  # Images don't have words

    def visit_table(self, element):
        for row in element.rows:
            self.word_count += len(row)

if __name__ == "__main__":
    # Define document elements
    doc = [
        Paragraph("This is a paragraph with some words."),
        Image("cover.jpg"),
        Table([["Name", "Age"], ["Alice", "30"], ["Bob", "25"]])
    ]

    # Apply WordCounter visitor
    counter = WordCounter()

    for element in doc:
        element.accept(counter)

    print(f"Total word count: {counter.word_count}")

Total word count: 13
